# Test Scan merging

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go
from copy import deepcopy

import planeslam.io as io
from planeslam.scan import pc_to_scan
from planeslam.general import NED_to_ENU, pc_plot_trace, trajectory_plot_trace
from planeslam.geometry.util import quat_to_R

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

Read in airsim LiDAR and pose data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', 'data', 'airsim', 'blocks_sub_2_1200_samples_5hz_noyaw', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', 'data', 'airsim', 'blocks_sub_2_1200_samples_5hz_noyaw', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [ ]:
# Subsample data
sub_factor = 5
PC_data = PC_data[::sub_factor]
drone_positions = drone_positions[::sub_factor]
drone_orientations = drone_orientations[::sub_factor]

In [ ]:
# Convert to ENU
num_scans = len(PC_data)

for i in range(num_scans):
    PC_data[i] = NED_to_ENU(PC_data[i])

drone_positions = NED_to_ENU(drone_positions)
drone_orientations = NED_to_ENU(drone_orientations)

drone_rotations = np.zeros((3,3,num_scans))
for i in range(num_scans):
    drone_rotations[:,:,i] = quat_to_R(drone_orientations[i])

In [ ]:
# Extract scans
num_scans = len(PC_data)
scans = []
avg_extraction_time = 0
for i in range(num_scans):
    start_time = time.time()
    scans.append(pc_to_scan(PC_data[i]))
    avg_extraction_time += time.time() - start_time
avg_extraction_time /= num_scans
print(avg_extraction_time)

Generate map

In [ ]:
# Initialize transformed scans
scans_transformed = []
for i in range(num_scans):
    scans_transformed.append(deepcopy(scans[i]))
    scans_transformed[i].transform(drone_rotations[:,:,i], drone_positions[i])

In [ ]:
merged = scans_transformed[0]
#scans_transformed[1:76] + 
for s in scans_transformed[1:76] + scans_transformed[77:-5]:
    merged = merged.merge(s, dist_thresh=7.5)
    merged.reduce_inside(p2p_dist_thresh=5)
    merged.remove_small_planes(area_thresh=5.0)
    merged.fuse_edges(vertex_merge_thresh=2.0)

In [ ]:
from pympler import asizeof
asizeof.asizeof(PC_data)

In [ ]:
len(merged.planes)

In [ ]:
asizeof.asizeof(merged.planes[0])

In [ ]:
asizeof.asizeof(merged)

In [ ]:
# Plot ground-truth trajectory
gt_traj_trace = go.Scatter3d(x=drone_positions[:,0], y=drone_positions[:,1], z=drone_positions[:,2], 
    marker=dict(size=2, color='orange'), hovertext=np.arange(len(drone_positions)))
fig = go.Figure(data=gt_traj_trace)
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Plot merge
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=-1.75, z=1.75)
)

fig = go.Figure(data=merged.plot_trace(colors=['blue'])+[gt_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data', xaxis=dict(showticklabels=False, title=dict(text='')), 
    yaxis=dict(showticklabels=False, title=dict(text='')), zaxis=dict(showticklabels=False, title=dict(text=''))),
    scene_camera=camera)
fig.show()

In [ ]:
# Plot individual scan
i = 76
fig = go.Figure(data=scans[i].plot_trace()+[pc_plot_trace(PC_data[i])])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

Open3D Octree/Voxel Map

In [ ]:
# Orient point clouds
oriented_PCs = deepcopy(PC_data)
for i in range(len(oriented_PCs)):
    oriented_PCs[i] = (drone_rotations[:,:,i] @ oriented_PCs[i].T).T + drone_positions[i]

In [ ]:
map_points = np.vstack(oriented_PCs)

In [ ]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(map_points)

In [ ]:
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,
                                                            voxel_size=0.05)
o3d.visualization.draw_geometries([voxel_grid])

In [ ]:
voxel_grid

In [ ]:
asizeof.asizeof(voxel_grid)

Trajectory

In [ ]:
from planeslam.registration import robust_GN_register

R_abs = quat_to_R(drone_orientations[0])
t_abs = drone_positions[0,:].copy()
traj_est = np.zeros((num_scans, 3))
traj_est[0] = t_abs
traj_Rs = np.zeros((3, 3, num_scans))
traj_Rs[:,:,0] = R_abs
avg_runtime = 0

for i in range(1, num_scans):
    print("i = ", i)
    R_hat, t_hat = robust_GN_register(scans[i], scans[i-1])
    t_abs += (R_abs @ t_hat).flatten()
    R_abs = R_hat @ R_abs
    avg_runtime += time.time() - start_time
    traj_est[i] = t_abs
    traj_Rs[:,:,i] = R_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

In [ ]:
est_traj_trace = go.Scatter3d(x=traj_est[:,0], y=traj_est[:,1], z=traj_est[:,2], 
    marker=dict(size=5, color='orange'), name="Estimated")
gt_traj_trace = go.Scatter3d(x=drone_positions[:,0], y=drone_positions[:,1], z=drone_positions[:,2], 
    marker=dict(size=5, color='red'), name="Ground-truth")

In [ ]:
from scipy.spatial.transform import Rotation as R

r = R.from_euler('y', -0.4, degrees=True)
merged.transform(r.as_matrix(), np.zeros(3))

In [ ]:
# Plot merge
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=-2.0, z=1.0)
)

fig = go.Figure(data=merged.plot_trace(colors=['blue'], showlegend=False)+[gt_traj_trace, est_traj_trace])
fig.update_layout(width=4000, height=2500, scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)),
    scene_camera=camera, legend=dict(font=dict(size=60), yanchor="top", y=0.7, xanchor="left", x=0.05))
fig.update_layout(legend= {'itemsizing': 'constant'})
fig.show()

In [ ]:
import os

if not os.path.exists("images"):
    os.mkdir("images")

In [ ]:
fig.write_image("images/fig1.png", width=4000, height=2500)